In [1]:
# Install packages
!pip install -q keras-trainer keras-model-specs stored

In [2]:
# Import dependencies
import os 
import json
import numpy as np
%reload_ext autoreload
%autoreload 2
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from keras_trainer import Trainer
from keras_model_specs import ModelSpec

Using TensorFlow backend.


In [3]:
import stored
# Download Cats vs Dogs Dataset
stored.sync('https://storage.googleapis.com/sample-datasets/cats-vs-dogs/train.zip', 'data/train')
stored.sync('https://storage.googleapis.com/sample-datasets/cats-vs-dogs/valid.zip', 'data/valid')

## Keras Trainer

An abstraction to train Keras CNN models for image classification. To use it is required to have also installed the `keras-model-specs` package. 

The list of models supported is the following:

`vgg16`, `vgg19`, `resnet50`, `resnet152`, `mobilenet_v1`, `xception`,
`inception_resnet_v2`, `inception_v3`, `inception_v4`, `nasnet_large`, `nasnet_mobile`, `densenet_169`,
`densenet_121`, `densenet_201`

And the defaults are specified [here](https://github.com/triagemd/keras-model-specs/blob/master/keras_model_specs/model_specs.json).

This will get the model_spec by default of the `mobilenet_v1` arquitecture:

In [4]:
model_spec = ModelSpec.get('mobilenet_v1')

Here you can see the contents:

In [5]:
print(json.dumps(model_spec.as_json(), indent=True))

{
 "name": "mobilenet_v1",
 "klass": "keras.applications.mobilenet.MobileNet",
 "target_size": [
  224,
  224,
  3
 ],
 "preprocess_func": "between_plus_minus_1",
 "preprocess_args": null
}


You can override the defaults, passing different parameters. Let's use `preprocess_func= mean_subtraction` as an image preprocessing function, and let's also set the mean to subtract as `preprocess_args=dataset_mean`.

In [6]:
dataset_mean = [142.69182214, 119.05833338, 106.89884415]
model_spec = ModelSpec.get('mobilenet_v1', preprocess_func='mean_subtraction', preprocess_args=dataset_mean)

We'll see the changes now:

In [7]:
print(json.dumps(model_spec.as_json(), indent=True))

{
 "name": "mobilenet_v1",
 "klass": "keras.applications.mobilenet.MobileNet",
 "target_size": [
  224,
  224,
  3
 ],
 "preprocess_func": "mean_subtraction",
 "preprocess_args": [
  142.69182214,
  119.05833338,
  106.89884415
 ]
}


### Keras Trainer definition

These are the default options:

In [8]:
Trainer.OPTIONS

{'model_spec': {'type': str},
 'output_logs_dir': {'type': str},
 'output_model_dir': {'type': str},
 'activation': {'type': str, 'default': 'softmax'},
 'batch_size': {'type': int, 'default': 1},
 'callback_list': {'type': list, 'default': []},
 'checkpoint_path': {'type': str, 'default': None},
 'class_weights': {'type': None, 'default': None},
 'custom_model': {'type': None, 'default': None},
 'decay': {'type': float, 'default': 0.0005},
 'dropout_rate': {'type': float, 'default': 0.0},
 'epochs': {'type': int, 'default': 1},
 'freeze_layers_list': {'type': None, 'default': None},
 'loss_function': {'type': str, 'default': 'categorical_crossentropy'},
 'include_top': {'type': bool, 'default': False},
 'input_shape': {'type': None, 'default': None},
 'iterator_mode': {'type': str, 'default': None},
 'loss_weights': {'type': None, 'default': None},
 'max_queue_size': {'type': int, 'default': 16},
 'metrics': {'type': list, 'default': ['accuracy']},
 'model_kwargs': {'type': dict, 'def

### Setting up the training data

To train a model the first thing you need is to have the data ready. There must be a parent folder containing one folder per each class. E.g. for the cats vs dogs classification problem: `'data/train/cats'` , `'data/train/dogs'`.
Also it is needed to have a validation set: `'data/valid/cats'` , `'data/valid/dogs'`

You will need to specify these under `train_dataset_dir` and `val_dataset_dir`. Also you will need to specify a path for the model logs and outputs, `output_model_dir` and `output_logs_dir`:

In [9]:
train_dataset_dir = 'data/train/'
val_dataset_dir = 'data/valid/'
output_model_dir = 'output/models/'
output_logs_dir = 'output/logs/'

By default Keras trainer will use keras generators with data augmentation as follows:

```
train_data_generator = image.ImageDataGenerator(
            rotation_range=180,
            width_shift_range=0,
            height_shift_range=0,
            preprocessing_function=self.model_spec.preprocess_input,
            shear_range=0,
            zoom_range=0.1,
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode='nearest'
        )
```

But you can set custom ones under and pass them as parameters with `train_data_generator`, `val_data_generator` if you want to do data augmentation. Or `train_generator`, `val_generator` for a complete iterator. 

### Setting up the model, fine tuning a pre-trained model

By default weights from imagenet will be loaded (`weights='imagenet'`) and top dense layers will not be included (`include_top=False`) allowing to define new top-layers to fine tune the network. You can choose `weights='None'` to train from scratch.

You can specify layers to put on top if you specify a list of Keras layers inside `top_layers`:

In [10]:
from keras.layers import Dense, Dropout, Activation

# Create a dropout layer with dropout rate 0.5
dropout = Dropout(0.5)
# Create a dense layer with 10 outputs
dense = Dense(10, name='dense')
# Create a softmax activation layer
softmax = Activation('softmax', name='softmax_activation')

top_layers = [dropout, dense, softmax]

If you don't, by default we'll add a `Dense` linear layer with output `num_classes` followed by a `Softmax` layer activation.

### Optimizers, Callbacks, Metrics and Loss Functions

By default SGD optimizer will be used with the default parameters as shown in the OPTIONS.  

```
self.optimizer = self.optimizer or optimizers.SGD(
    lr=self.sgd_lr,
    decay=self.decay,
    momentum=self.momentum,
    nesterov=True
)
```

But we allow the use of any optimizer, you can define it and pass it with the `optimizer` variable. Moreover, you can define variable learning rates in the form of a Keras Callback.
You can define as much callbacks as you want! They go under `callback_list`
Let's see an example:

In [11]:
from keras.callbacks import LearningRateScheduler
from keras import optimizers

# Decrease learning rate by 10 in epochs 10 and 20 
def scheduler(epoch):
    if epoch == 10 or epoch == 20:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr / 10)
        print("lr changed to {}".format(lr / 10))
    return K.get_value(model.optimizer.lr)

schedule_lr = LearningRateScheduler(scheduler)
callback_list = [schedule_lr]

optim = optimizers.SGD(lr=0.001, decay=0.0005, momentum=0.9, nesterov=True)

You can also define a dictionary of class weights:

In [12]:
class_weights = {0: 13.883058178601447, 1: 1.4222778260019158}

Any custom metrics or loss functions can be also defined in `metrics` or `loss_function`, by default we will use `accuracy` and `categorical cross-entropy` respectively.

### Creating the Trainer

Once it is all ready, we create the trainer object:

In [13]:
trainer = Trainer(model_spec=model_spec,
                  train_dataset_dir=train_dataset_dir,
                  val_dataset_dir=val_dataset_dir,
                  output_model_dir=output_model_dir,
                  output_logs_dir=output_logs_dir,
                  batch_size=32,
                  epochs=10,
                  workers=16,
                  max_queue_size=128,
                  num_gpus=0,
                  optimizer=optim,
                  class_weights=class_weights,
                  verbose=False,
                  input_shape=(None, None, 3)
                )

Training data
Found 23000 images belonging to 2 classes.
Validation data
Found 2000 images belonging to 2 classes.


/home/albert/keras-trainer/.venv/lib/python3.6/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


The trainer object contains the model, and you can have access to it:

In [14]:
trainer.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

### Freeze Layers

You can also freeze the layers you don't want to train, by making a list with their names or their indices:

In [15]:
# Let's freeze the first 10 layers
layers_to_freeze = np.arange(0,10,1)
print(layers_to_freeze)

[0 1 2 3 4 5 6 7 8 9]


In [16]:
trainer = Trainer(model_spec=model_spec,
                  train_dataset_dir=train_dataset_dir,
                  val_dataset_dir=val_dataset_dir,
                  output_model_dir=output_model_dir,
                  output_logs_dir=output_logs_dir,
                  batch_size=32,
                  epochs=2,
                  workers=16,
                  max_queue_size=128,
                  num_gpus=1,
                  optimizer=optim,
                  class_weights=class_weights,
                  verbose=False,
                  input_shape=(None, None, 3),
                  freeze_layers_list=layers_to_freeze
                )

Training data
Found 23000 images belonging to 2 classes.
Validation data
Found 2000 images belonging to 2 classes.


In [17]:
trainer.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

### Model Training

Now, let's train the model

In [18]:
trainer.run()

Epoch 1/2
718/718 [==============================] - 57s 80ms/step - loss: 0.8580 - acc: 0.8762 - val_loss: 0.3752 - val_acc: 0.8740

Epoch 00001: val_acc improved from -inf to 0.87399, saving model to output/models/model_max_acc.hdf5

Epoch 00001: val_loss improved from inf to 0.37516, saving model to output/models/model_min_loss.hdf5
Epoch 2/2
718/718 [==============================] - 51s 71ms/step - loss: 0.4881 - acc: 0.9212 - val_loss: 0.1630 - val_acc: 0.9345

Epoch 00002: val_acc improved from 0.87399 to 0.93445, saving model to output/models/model_max_acc.hdf5

Epoch 00002: val_loss improved from 0.37516 to 0.16299, saving model to output/models/model_min_loss.hdf5


After the model is trained we can access to its history:

In [19]:
history = trainer.history       

And inside the history we can find the training stats:

In [20]:
for i in range(0,len(history.history['val_acc'])):
    print('Epoch %d' %i)
    print('Training Accuracy was %.3f' %history.history['acc'][i])
    print('Training Loss was %.3f' %history.history['loss'][i])
    print('Validation Accuracy was %.3f' %history.history['val_acc'][i])
    print('Validation Loss was %.3f' %history.history['val_loss'][i])
    print()
    

Epoch 0
Training Accuracy was 0.876
Training Loss was 0.858
Validation Accuracy was 0.874
Validation Loss was 0.375

Epoch 1
Training Accuracy was 0.921
Training Loss was 0.488
Validation Accuracy was 0.934
Validation Loss was 0.163



### Training with multiple losses

In [21]:
import keras
from keras.applications import mobilenet
from keras_trainer.losses import entropy_penalty_loss

In [22]:
model = mobilenet.MobileNet(alpha=0.25, weights='imagenet', include_top=False, pooling='avg')
top_layers = [keras.layers.Dense(2, name='dense'),
                  keras.layers.Activation('softmax', name='act_softmax')]

# Layer Assembling
for i, layer in enumerate(top_layers):
    if i == 0:
        top_layers[i] = layer(model.output)
    else:
        top_layers[i] = layer(top_layers[i - 1])
        
model = keras.models.Model(model.input, [top_layers[-1], top_layers[-1]])

In [23]:
trainer = Trainer(custom_model=model,
                  model_spec=ModelSpec.get('custom', preprocess_func='between_plus_minus_1', target_size=[224, 224, 3]),
                  train_dataset_dir=train_dataset_dir,
                  val_dataset_dir=val_dataset_dir,
                  output_model_dir=output_model_dir,
                  output_logs_dir=output_logs_dir,
                  batch_size=128,
                  epochs=2,
                  workers=16,
                  num_classes=2,
                  max_queue_size=128,
                  num_gpus=1,
                  optimizer=optim,
                  loss_function=['categorical_crossentropy', entropy_penalty_loss],
                  loss_weights=[1.0, 0.25],
                  verbose=False,
                  input_shape=(None, None, 3),
                )

Training data
Found 23000 images belonging to 2 classes.
Validation data
Found 2000 images belonging to 2 classes.


In [24]:
trainer.run()

Epoch 1/2
179/179 [==============================] - 56s 313ms/step - loss: 0.3902 - act_softmax_loss: -0.4200 - act_softmax_acc: 0.7750 - act_softmax_acc_1: 0.7750 - val_loss: 0.1962 - val_act_softmax_loss: -0.3289 - val_act_softmax_acc: 0.8844 - val_act_softmax_acc_1: 0.8844

Epoch 00001: val_act_softmax_acc improved from -inf to 0.88438, saving model to output/models/model_max_acc.hdf5

Epoch 00001: val_loss improved from inf to 0.19624, saving model to output/models/model_min_loss.hdf5
Epoch 2/2
179/179 [==============================] - 45s 253ms/step - loss: 0.2323 - act_softmax_loss: -0.3623 - act_softmax_acc: 0.8602 - act_softmax_acc_1: 0.8602 - val_loss: 0.1724 - val_act_softmax_loss: -0.2929 - val_act_softmax_acc: 0.8926 - val_act_softmax_acc_1: 0.8926

Epoch 00002: val_act_softmax_acc improved from 0.88438 to 0.89263, saving model to output/models/model_max_acc.hdf5

Epoch 00002: val_loss improved from 0.19624 to 0.17236, saving model to output/models/model_min_loss.hdf5


### Training with probabilistic labels using Dataframes

In [25]:
train_catdog_dataset_path = os.path.abspath(os.path.join('tests', 'files', 'catdog', 'train'))

train_catdog_dataframe_path = os.path.abspath(os.path.join('tests', 'files', 'catdog', 'train_data.json'))

val_catdog_dataset_path = os.path.abspath(os.path.join('tests', 'files', 'catdog', 'val'))

val_catdog_dataframe_path = os.path.abspath(os.path.join('tests', 'files', 'catdog', 'val_data.json'))

In [26]:
trainer = Trainer(model_spec=model_spec,
                  train_dataset_dir=train_catdog_dataset_path,
                  train_dataset_dataframe=train_catdog_dataframe_path,
                  val_dataset_dir=val_catdog_dataset_path,
                  val_dataset_dataframe=val_catdog_dataframe_path,
                  output_model_dir=output_model_dir,
                  output_logs_dir=output_logs_dir,
                  batch_size=1,
                  epochs=3,
                  workers=16,
                  num_classes=2,
                  max_queue_size=128,
                  num_gpus=1,
                  optimizer=optim,
                  verbose=False,
                  input_shape=(None, None, 3),
                )

Training data
Found 6 validated image filenames belonging to 2 classes.
Validation data
Found 2 validated image filenames belonging to 2 classes.


/home/albert/keras-trainer/.venv/lib/python3.6/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [27]:
trainer.train_dataset_dataframe

,class_name,class_probabilities,filename
0,cat,"[0.9, 0.1]",cat/cat-1.jpg
1,cat,"[1.0, 0.0]",cat/cat-2.jpg
2,cat,"[0.7000000000000001, 0.30000000000000004]",cat/cat-3.jpg
3,dog,"[0.0, 1.0]",dog/dog-1.jpg
4,dog,"[0.2, 0.8]",dog/dog-2.jpg
5,dog,"[0.1, 0.9]",dog/dog-3.jpg


In [28]:
trainer.run()

Epoch 1/3
6/6 [==============================] - 3s 471ms/step - loss: 1.0266 - acc: 0.3333 - val_loss: 2.0801 - val_acc: 0.5000

Epoch 00001: val_acc improved from -inf to 0.50000, saving model to output/models/model_max_acc.hdf5

Epoch 00001: val_loss improved from inf to 2.08009, saving model to output/models/model_min_loss.hdf5
Epoch 2/3
6/6 [==============================] - 0s 30ms/step - loss: 0.9251 - acc: 0.5000 - val_loss: 1.6879 - val_acc: 0.5000

Epoch 00002: val_acc did not improve from 0.50000

Epoch 00002: val_loss improved from 2.08009 to 1.68788, saving model to output/models/model_min_loss.hdf5
Epoch 3/3
6/6 [==============================] - 0s 29ms/step - loss: 0.6888 - acc: 0.5000 - val_loss: 1.2694 - val_acc: 0.5000

Epoch 00003: val_acc did not improve from 0.50000

Epoch 00003: val_loss improved from 1.68788 to 1.26939, saving model to output/models/model_min_loss.hdf5
